In [ ]:
from optimisation_models.prosumer_rho_model import DataPreparation, ProsumerModel, ProsumerRho

In [ ]:
solar_home_file='data/raw/solar_home_processed_in_sample.csv'
price_file = 'data/predispatch_prices/PD_NSW_2021.csv'

In [ ]:
for optimised_month in range(1, 13):
    if optimised_month == 7:
        continue
    data_preparation = DataPreparation(solar_home_file, price_file)
    df_prosumer = data_preparation.get_prosumer_data(month=optimised_month)
    df_price = data_preparation.get_pd_price_data(month=optimised_month)

    prosumer_model = ProsumerModel(df_prosumer, df_price)
    rolling_model = ProsumerRho(prosumer_model)

    # Run this to get the prosumer rho
    dfs_rolling_interval = rolling_model.rolling_optimisation(n_receding_horizons=48*7)

    folder_name = f'data/processed/prosumer_rho/month_{optimised_month}'
    rolling_model.save_rolling_data(folder_name)

In [ ]:
for optimised_month in range(1, 13):
    dfs_agg_rolling = rolling_model.compile_aggregate_data(f'data/processed/prosumer_rho/month_{optimised_month}')
    agg_folder_name = f'data/processed/prosumer_rho/aggregate_data/month_{optimised_month}'
    rolling_model.save_aggregate_rolling_data(agg_folder_name)